In [1]:
import zipfile
import os
zip_file_path = '/content/PySpark, PySQL notebook - Raahul.zip'
output_directory = 'content'
os.makedirs(output_directory, exist_ok=True)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_directory)

In [2]:
!pip install pyspark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RealEstate").getOrCreate()


In [4]:
real_estate_df = spark.read.csv('/content/content/real_estate.csv', header=True, inferSchema=True)


In [11]:
df = spark.read.csv("/content/content/real_estate.csv", header=True, inferSchema=True)

In [13]:
df.printSchema()

root
 |-- Address: string (nullable = true)
 |-- Rooms: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Bedroom: integer (nullable = true)
 |-- Bathroom: integer (nullable = true)
 |-- Landsize: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Type: string (nullable = true)



In [14]:
df.createOrReplaceTempView("real_estate")

In [6]:
print(list(real_estate_df.columns))

['Address', 'Rooms', 'Price', 'Bedroom', 'Bathroom', 'Landsize', 'YearBuilt', 'Regionname', 'Suburb', 'Type']


### Number of houses built before 2000

In [7]:
houses_before_2000 = real_estate_df.filter(real_estate_df['YearBuilt'] < 2000).count()
print("Number of houses built before 2000:", houses_before_2000)


Number of houses built before 2000: 6169


In [15]:
houses_before_2000_query = """
SELECT COUNT(*) AS NumberOfHousesBefore2000
FROM real_estate
WHERE YearBuilt < 2000
"""
houses_before_2000_result = spark.sql(houses_before_2000_query)
houses_before_2000_result.show()

+------------------------+
|NumberOfHousesBefore2000|
+------------------------+
|                    6169|
+------------------------+



### Number of houses whose price is greater than the average price

In [8]:
average_price = real_estate_df.agg({'Price': 'avg'}).collect()[0][0]

# Count houses whose price is greater than the average price
houses_above_average_price = real_estate_df.filter(real_estate_df['Price'] > average_price).count()
print("Number of houses with price greater than average:", houses_above_average_price)


Number of houses with price greater than average: 5186


In [16]:
above_average_price_query = """
WITH AvgPrice AS (
    SELECT AVG(Price) AS AveragePrice
    FROM real_estate
)
SELECT COUNT(*) AS NumberOfHousesAboveAverage
FROM real_estate
WHERE Price > (SELECT AveragePrice FROM AvgPrice)
"""
above_average_price_result = spark.sql(above_average_price_query)
above_average_price_result.show()

+--------------------------+
|NumberOfHousesAboveAverage|
+--------------------------+
|                      5186|
+--------------------------+



### Average price of a home with Regionname = "Southern Metropolitan" and Suburb="Albert Park"

In [9]:
average_price_albert_park = real_estate_df.filter(
    (real_estate_df['Regionname'] == "Southern Metropolitan") &
    (real_estate_df['Suburb'] == "Albert Park")
).agg({'Price': 'avg'}).collect()[0][0]

print("Average price of home in Southern Metropolitan, Albert Park:", average_price_albert_park)

Average price of home in Southern Metropolitan, Albert Park: 1941355.072463768


In [17]:
average_price_query = """
SELECT AVG(Price) AS AveragePrice
FROM real_estate
WHERE Regionname = 'Southern Metropolitan' AND Suburb = 'Albert Park'
"""
average_price_result = spark.sql(average_price_query)
average_price_result.show()

+-----------------+
|     AveragePrice|
+-----------------+
|1941355.072463768|
+-----------------+

